# Verify ArUco Package Integration

This notebook verifies the `snap_fit.aruco` and `snap_fit.puzzle.sheet_aruco` modules.


## Import


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from pathlib import Path
import sys

import cv2
import matplotlib.pyplot as plt
import numpy as np

from snap_fit.aruco.aruco_board import ArucoBoardGenerator
from snap_fit.aruco.aruco_detector import ArucoDetector
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.config.aruco.sheet_aruco_config import SheetArucoConfig
from snap_fit.puzzle.sheet_aruco import SheetAruco

In [ ]:
from snap_fit.config.types import EDGE_ENDS_TO_CORNER
from snap_fit.config.types import EdgePos
from snap_fit.image.process import find_contours
from snap_fit.image.process import find_corners
from snap_fit.image.segment_matcher import SegmentMatcher
from snap_fit.image.utils import draw_contour
from snap_fit.image.utils import draw_corners
from snap_fit.image.utils import show_image_mpl
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.piece import Piece
from snap_fit.puzzle.sheet import Sheet

In [ ]:
snap_fit_paths = get_snap_fit_paths()

## Load Image Sheet


In [ ]:
# sheets_tag = "oca"
sheets_tag = "milano1"

In [ ]:
sheets_base_fol = snap_fit_paths.data_fol / sheets_tag
img_fol = sheets_base_fol / "sheets"

In [ ]:
# save the config once
board_config = ArucoBoardConfig(
    markers_x=5,
    markers_y=7,
    marker_length=100,
    marker_separation=100,
)
detector_config = ArucoDetectorConfig(board=board_config)
sheet_config = SheetArucoConfig(
    min_area=80_000,
    detector=detector_config,
)


In [ ]:
sheet_aruco = SheetAruco(sheet_config)

## Load some image


In [ ]:
img_idx = 0
img_fp = list(img_fol.iterdir())[img_idx]
lg.debug(f"Testing on image: {img_fp}")

In [ ]:
sheet = sheet_aruco.load_sheet(img_fp)

plt.figure(figsize=(5, 7))
plt.imshow(sheet.img_orig, cmap="gray")
plt.title("SheetAruco Loaded Image")
plt.axis("off")
plt.show()

In [ ]:
piece = sheet.pieces[0]

img_contour = draw_contour(piece.img_bw, piece.contour_loc, color=127)
corners = list(piece.corners.values())
img_corners = draw_corners(img_contour, corners, color=190)
show_image_mpl(img_corners)